In [1]:
!pip install sentence-transformers faiss-cpu transformers datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 82.9 MB/s eta 0:00:00


In [2]:
import json
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [8]:
def extract_articles_v2(dataset):
    articles = []

    for category, subcats in dataset.items():
        for subcat, systems in subcats.items():
            for system_name, system_data in systems.items():

                brief = system_data.get("brief", "")
                metadata = system_data.get("metadata", {})

                parts = system_data.get("parts", {})
                for part_name, part_articles in parts.items():
                    for article in part_articles:
                        articles.append({
                            "category": category,
                            "sub_category": subcat,
                            "system": system_name,
                            "part": part_name,
                            "brief": brief,
                            "metadata": metadata,
                            "id": article.get("id"),
                            "title": article.get("Article_Title"),
                            "status": article.get("status"),
                            "text": article.get("Article_Text")
                        })
    return articles


In [12]:
with open("saudi_laws_scraped.json", "r", encoding="utf-8") as f:
    data = json.load(f)

articles = extract_articles_v2(data)
print(f"✅ Total Articles Extracted: {len(articles)}")
print(json.dumps(articles[604], indent=2, ensure_ascii=False))

✅ Total Articles Extracted: 16371
{
  "category": "أنظمة عادية",
  "sub_category": "الأمن الداخلي والأحوال المدنية والأنظمة الجنائية",
  "system": "نظام مكافحة غسل الأموال",
  "part": "main",
  "brief": "يتضمن النظام:\r\nالمقصود بالعبارات والألفاظ الواردة بالنظام. الأفعال التي يعد مرتكبها مرتكبًا جريمة غسل الأموال – ما يجب على المؤسسات المالية وغير المالية اتخاذه من إجراءات حيال مرتكب جريمة غسل الأموال – البرامج التي تضعها المؤسسات المالية وغير المالية لمكافحة عمليات غسل الأموال – وحدة مكافحة غسل الأموال – عقوبة مرتكب جريمة غسل الأموال.",
  "metadata": {
    "الاسم": "نظام مكافحة غسل الأموال",
    "تاريخ الإصدار": "1433/05/11 هـ  الموافق : 03/04/2012 مـ",
    "تاريخ النشر": "1433/08/02  هـ الموافق : 22/06/2012 مـ",
    "الحالة": "لاغي",
    "أدوات إصدار النظام": [
      {
        "text": "مرسوم ملكي رقم م / 31 بتاريخ 11 / 5 / 1433",
        "url": "https://laws.boe.gov.sa/BoeLaws/Laws/Viewer/9ec732e6-9bbf-4fda-8a61-a9ae00c3c014?lawId=4a8842df-9cd1-4ee7-bf97-a9a700f180d4"
      },
     

In [9]:
def build_corpus(articles):
    corpus = []
    for art in articles:
        meta = art.get("metadata", {})
        meta_str = " ".join([f"{k}: {v}" for k, v in meta.items() if v])
        item_text = f"{art['title']} - {art['text']} {meta_str}"
        corpus.append(item_text.strip())
    return corpus

corpus = build_corpus(articles)
print(f"✅ Corpus built with {len(corpus)} documents")
for i in range(2):
    print(f"\n--- Example {i+1} ---\n{corpus[i][:400]}...")

✅ Corpus built with 16371 documents

--- Example 1 ---
المادة الأولى - المملكة العربية السعودية، دولة عربية إسلامية، ذات
سيادة تامة
، دينها
الإسلام
، ودستورها
كتاب الله تعالى
وسنة رسوله صلى الله عليه وسلم. ولغتها هي اللغة العربية، وعاصمتها مدينة الرياض. الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ الموافق : 06/03/1992 مـ الحالة: ساري أدوات إصدار النظام: [{'text': 'أمر ملكي رقم أ/90 بتاري...

--- Example 2 ---
المادة الثانية - عيدا الدولة، هما عيدا الفطر والأضحى، وتقويمها، هو
التقويم الهجري. الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ الموافق : 06/03/1992 مـ الحالة: ساري أدوات إصدار النظام: [{'text': 'أمر ملكي رقم أ/90 بتاريخ 27 / 8 / 1412', 'url': 'https://laws.boe.gov.sa/BoeLaws/Laws/Viewer/0c1fa9f6-703e-4a93-ab8b-a6d7ad40628b?lawId=16b...


In [27]:
import torch

embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
embeddings = []
for i in tqdm(range(0, len(corpus), batch_size)):
    batch = corpus[i:i+batch_size]
    emb = embed_model.encode(batch, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=False)
    embeddings.append(emb)
embeddings = np.vstack(embeddings)


100%|██████████| 512/512 [41:04<00:00,  4.81s/it]


In [28]:
print(f"✅ Embeddings shape: {embeddings.shape}")

✅ Embeddings shape: (16371, 1024)


In [29]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

faiss.write_index(index, "m3V2_legal_faiss.index")

In [14]:
index = faiss.read_index("m3_legal_faiss.index")

In [18]:
def retrieve(query, top_k=5):
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, top_k)
    results = [(corpus[i], float(D[0][j])) for j, i in enumerate(I[0])]
    return results

query = "ما هو شعار الدولة السعودية؟"
results = retrieve(query)

for i, (text, score) in enumerate(results, 1):
    print(f"\n🔹 Result {i} (score={score:.3f})\n{text[:400]}...")


🔹 Result 1 (score=0.578)
المادة الأولى - المملكة العربية السعودية، دولة عربية إسلامية، ذات
سيادة تامة
، دينها
الإسلام
، ودستورها
كتاب الله تعالى
وسنة رسوله صلى الله عليه وسلم. ولغتها هي اللغة العربية، وعاصمتها مدينة الرياض. الاسم: النظام الأساسي للحكم تاريخ الإصدار: 1412/08/27 هـ  الموافق : 01/03/1992 مـ تاريخ النشر: 1412/09/02  هـ الموافق : 06/03/1992 مـ الحالة: ساري أدوات إصدار النظام: [{'text': 'أمر ملكي رقم أ/90 بتاري...

🔹 Result 2 (score=0.571)
المادة الثانية ( علم جلالة الملك ) - يكون لجلالة الملك علم خاص يُطابق
العلم الوطني
في أوصافِه ويُطرَّز في الزاوية الدُنيا مِنه المجاورة لعود العلم بخيوط حريرية مُذهبة   شعار الدولة     وهو السيفان المُتقاطِعان تعلوهما نخلة. وذلك وِفق النموذج المرفق رقم (2). الاسم: نظام العلم للمملكة العربية السعودية تاريخ الإصدار: 1393/01/01 هـ  الموافق : 04/02/1973 مـ تاريخ النشر: 1393/01/01  هـ الموافق : 04/02/1...

🔹 Result 3 (score=0.570)
المادة الرابعة - شعار الدولة
سيفان متقاطعان، ونخلة وسط فراغهما الأعلى، ويحدد النظام
نشيد الدولة
وأوسمتها. الاسم: ا

In [ ]:
from sentence_transformers import CrossEncoder

reranker_model_name = "cross-encoder/ms-marco-TinyBERT-L-2"
reranker = CrossEncoder(reranker_model_name)

def rerank_results(query, results, top_k=5):

    pairs = [[query, r["chunk"]] for r in results]

    scores = reranker.predict(pairs)
    reranked_results = [results[i] for i in np.argsort(scores)[::-1]]

    return reranked_results[:top_k]

query = "أنواع لوحات المركبات؟"
initial_results = retrieve(query, top_k=10)
reranked_results = rerank_results(query, initial_results)

print("Re-ranked Results:")
for r in reranked_results:
    print("📘", r["metadata"]["title"])
    print(r["chunk"])
    print("-" * 80)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Re-ranked Results:
📘 المادة الرابعة
يجب أن تحمل كل مركبة - فيما عدا الدراجة الآلية و المقطورة ونصف المقطورة - لوحتين ظاهرتين مقروءتين، توضحان رقم تسجيلهما، تكون إحداهما في مقدمة المركبة ، والأخرى في مؤخرتها، ولا يجوز سير المركبة دونهما.
--------------------------------------------------------------------------------
📘 جدول رسوم لوحات المركبات بأنواعها
العدد النوع رسم اللوحات رسم التالف والمفقود 1 لوحة سيارة خاصة 100 ريال 100 ريال 2 لوحة سيارة نقل خاص 100 ريال 100 ريال 3 لوحة حافلة خاصة 100 ريال 100 ريال 4 لوحة سيارة أجرة 100 ريال 100 ريال 5 لوحة سيارة نقل عام 100 ريال 100 ريال 6 لوحة حافلة عامة 100 ريال 100 ريال 7 لوحة دراجة آلية 100 ريال 100 ريال 8 لوحة مركبة أشغال عامة 100 ريال 100 ريال 9 لوحة مؤقتة 300 ريال 100 ريال 10 لوحة دبلوماسية أو قنصلية 100 ريال 100 ريال 11 لوحة تصدير 100 ريال 100 ريال 12 لوحة مقطورة أو نصف مقطورة 100 ريال 100 ريال
--------------------------------------------------------------------------------
📘 المادة السادسة
لا يجوز لأي مركبة حمل لوحات غير التي تصدر من الإ